In [1]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense # for creating regular densely-connected NN layers.
import tensorflow as tf
from keras.models import load_model

# Data manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Sklearn
import sklearn # for model evaluation
from sklearn.model_selection import train_test_split # for splitting data into train and test samples
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Erstellung eigener Aktivierungsfunktion
from keras import backend as K
from sklearn.ensemble import RandomForestRegressor

import os
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.impute import KNNImputer
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

ImportError: cannot import name '_check_weights' from 'sklearn.neighbors._base' (C:\Users\Ivan\anaconda3\lib\site-packages\sklearn\neighbors\_base.py)

In [ ]:
pip install scikit-learn==1.1.1

In [6]:
desired_cpu_usage = 0.90
num_threads = int(os.cpu_count() * desired_cpu_usage)

os.environ['OMP_NUM_THREADS'] = str(num_threads)
os.environ['TF_NUM_INTEROP_THREADS'] = str(num_threads)
os.environ['TF_NUM_INTRAOP_THREADS'] = str(num_threads)

tf.config.threading.set_inter_op_parallelism_threads(num_threads)
tf.config.threading.set_intra_op_parallelism_threads(num_threads)

RuntimeError: Inter op parallelism cannot be modified after initialization.

In [24]:
class DataProcessor:
    def __init__(self):
        self.raw_data = None
        self.X = None
        self.Y = None
        self.new_df = None
        self.data_inpute = None
        self.result = None

    def import_data(self, sample_size = 11500):
        csv_files = []
        for filename in os.listdir():
            if filename == "Datasets":
                for csv_file in os.listdir(filename):
                    if csv_file.endswith('.csv'):
                        csv_files.append(csv_file)

        self.raw_data = pd.read_csv(os.path.join("Datasets", csv_files[0]))
        self.raw_data.drop(["Time"], axis=1, inplace=True)
        self.raw_data.drop(["Amount"], axis=1, inplace=True)
        
        # Auswahl von 4000 zufälligen Zeilen
        self.raw_data = self.raw_data.sample(n=sample_size, random_state=42)        
        
        self.Y = self.raw_data["Class"]
        self.X = self.raw_data.drop(["Class"], axis=1)
        return self.raw_data, self.Y, self.X

    def gen_miss_values(self, p):
        # Auswahl von 35% zufälligen Zeilen aus X
        sample_percentage = 0.35
        sampled_rows = int(self.X.shape[0] * sample_percentage)
        sampled_X = self.X.sample(n=sampled_rows, random_state=42)
        shape = sampled_X.shape
        
        self.new_df = sampled_X.copy().astype(np.float64)
        missing = np.random.binomial(1, p, shape)
        self.new_df[missing.astype('bool')] = np.nan
        return self.new_df

    
    def print_progress(self, model, progress):
        print(f"Imputing {model} model: {progress * 100:.2f}% completed")
    
    def inpute_data(self, model):
        if model == "mean":
            self.data_inpute = self.new_df.fillna(self.new_df.mean())
            self.data_inpute = pd.concat([self.data_inpute, self.Y], axis=1, sort=False)
            columns = self.data_inpute.columns.tolist()

            for i in range(len(columns) - 1):
                columns[i] = "col_" + str(i + 1)
            self.data_inpute.columns = columns
            self.data_inpute.columns = [*self.data_inpute.columns[:-1], 'Y']
            
            self.Y = self.data_inpute["Y"]
            self.X = self.data_inpute.drop(["Y"], axis=1)
            return self.data_inpute

        elif model == "MICE":
            imputer = IterativeImputer()
            self.data_inpute = pd.DataFrame(imputer.fit_transform(self.new_df), columns=self.new_df.columns)
            self.data_inpute = pd.concat([self.data_inpute, self.Y], axis=1, sort=False)

            columns = self.data_inpute.columns.tolist()

            for i in range(len(columns) - 1):
                columns[i] = "col_" + str(i + 1)
            self.data_inpute.columns = columns
            self.data_inpute.columns = [*self.data_inpute.columns[:-1], 'Y']

            self.Y = self.data_inpute["Y"]
            self.X = self.data_inpute.drop(["Y"], axis=1)

            return self.data_inpute, self.raw_data["Class"]

        elif model == "kNN":
            imputer = KNNImputer()
            self.data_inpute = pd.DataFrame(imputer.fit_transform(self.new_df), columns=self.new_df.columns)
            self.data_inpute = pd.concat([self.data_inpute, self.Y], axis=1, sort=False)

            columns = self.data_inpute.columns.tolist()

            for i in range(len(columns) - 1):
                columns[i] = "col_" + str(i + 1)
            self.data_inpute.columns = columns
            self.data_inpute.columns = [*self.data_inpute.columns[:-1], 'Y']

            self.Y = self.data_inpute["Y"]
            self.X = self.data_inpute.drop(["Y"], axis=1)
            
            return self.data_inpute, self.raw_data["Class"]
        
        elif model == "RF":
            imputer = MissForest(max_iter=5, random_state=42, n_jobs=-1, criterion='squared_error')
            self.data_inpute = pd.DataFrame(imputer.fit_transform(self.new_df), columns=self.new_df.columns)
            self.data_inpute = pd.concat([self.data_inpute, self.Y], axis=1, sort=False)

            columns = self.data_inpute.columns.tolist()

            for i in range(len(columns) - 1):
                columns[i] = "col_" + str(i + 1)
            self.data_inpute.columns = columns
            self.data_inpute.columns = [*self.data_inpute.columns[:-1], 'Y']

            self.Y = self.data_inpute["Y"]
            self.X = self.data_inpute.drop(["Y"], axis=1)

            return self.data_inpute, self.raw_data["Class"]
    
    def model(self, model):
        if model == 0:
            return load_model("model_D0_01.h5")
        elif model == 1:
            return load_model("model_D0_02.h5")
        elif model == 2:
            return load_model("model_D0_03.h5")

    def evaluate(self, y_test, y_nan):
        y_true = y_test
        y_pred = y_nan
        conf_matrix = confusion_matrix(y_true, y_pred)

        # Compute the accuracy
        accuracy = accuracy_score(y_true, y_pred)

        # Compute the precision
        precision_scores = precision_score(y_true, y_pred, labels=range(len(conf_matrix)), average=None)

        # Compute the recall
        recall = recall_score(y_true, y_pred, labels=range(len(conf_matrix)), average=None)

        # Compute the F1-score
        f1 = f1_score(y_true, y_pred, labels=range(len(conf_matrix)), average=None)

        # Compute the average accuracy
        avg_accuracy = sum(precision_scores) / len(conf_matrix)

        # Compute the predicted probabilities
        y_scores = np.random.rand(len(y_true), len(np.unique(y_true)))

        ## Compute the AUC for each class
        n_classes = conf_matrix.shape[0]
        auc_list = []
        for i in range(n_classes):
            auc_list.append(roc_auc_score(y_true == i, y_scores[:, i]))

        # Define the result dictionary
        self.result = {
            "confusion_matrix": conf_matrix,
            "accuracy": accuracy,
            "precision": precision_scores,
            "recall": recall,
            "f1_score": f1,
            "avg_accuracy": avg_accuracy,
            "auc": auc_list
        }
        return self.result
    def save_txt(self, filename, evaluate):
        with open(filename, 'w') as f:
            f.write(str(evaluate))

In [5]:
data_processor = DataProcessor()
load_data = data_processor.import_data()
test_size = 0.3  # Konstante Testgröße

for model_number in range(0, 3):  # Modelle 1-3 durchlaufen
    model_name = f"Model_{model_number + 1}"
    output_prefix = f"prediction_ref_mean_D0_{model_name}_"
    model = data_processor.model(model_number)
    
    for missing_rate in [0.3, 0.6, 0.9]:
        miss_data = data_processor.gen_miss_values(missing_rate)
        inpute_values = data_processor.inpute_data("mean")
        
        comparison_col = [1 if x == y else 0 for x, y in zip(inpute_values["Y"], load_data[0]["Class"])]
        df = pd.DataFrame({"Y": inpute_values["Y"], "Class": load_data[0]["Class"], "Comparison": comparison_col})
        
        Y = inpute_values["Y"]
        X = inpute_values.drop(["Y"], axis=1)
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=42)

        y_pred = model.predict(X_test)
        df = pd.DataFrame(y_pred, index=X_test.index)
        df['larger_value'] = df.apply(lambda row: 1 if row[0] > row[1] else 0 if row[0] < row[1] else 0.5, axis=1)
        y_pred = df["larger_value"]

        result = pd.DataFrame()
        result["y_pred"] = y_pred
        result["y_test"] = y_test[X_test.index].values
        threshold = 0.5
        y_pred_binary = np.where(result['y_pred'] > threshold, 1, 0)
        y_test_binary = np.where(result['y_test'] > threshold, 1, 0)
        evaluate = data_processor.evaluate(y_test_binary, y_pred_binary)
        print(evaluate)
        filename = output_prefix + str(missing_rate).replace(".", "_")
        data_processor.save_txt(filename, evaluate)

108/108 [==============================] - 1s 3ms/step
{'confusion_matrix': array([[2232, 1212],
       [   4,    2]], dtype=int64), 'accuracy': 0.647536231884058, 'precision': array([0.99821109, 0.00164745]), 'recall': array([0.64808362, 0.33333333]), 'f1_score': array([0.78591549, 0.00327869]), 'avg_accuracy': 0.49992926884616856, 'auc': [0.4143921796360821, 0.49172473867595823]}
108/108 [==============================] - 0s 3ms/step
{'confusion_matrix': array([[2231, 1213],
       [   3,    3]], dtype=int64), 'accuracy': 0.647536231884058, 'precision': array([0.99865712, 0.00246711]), 'recall': array([0.64779326, 0.5       ]), 'f1_score': array([0.78584008, 0.00490998]), 'avg_accuracy': 0.5005621112707911, 'auc': [0.3512388695315524, 0.47706155632984903]}
108/108 [==============================] - 0s 3ms/step
{'confusion_matrix': array([[2231, 1216],
       [   3,    0]], dtype=int64), 'accuracy': 0.6466666666666666, 'precision': array([0.99865712, 0.        ]), 'recall': array([0.6

In [13]:
data_processor = DataProcessor()
load_data = data_processor.import_data()
test_size = 0.3  # Konstante Testgröße

for model_number in range(0, 3):  # Modelle 1-3 durchlaufen
    model_name = f"Model_{model_number + 1}"
    output_prefix = f"prediction_ref_MICE_D0_{model_name}_"
    model = data_processor.model(model_number)
    
    for missing_rate in [0.3, 0.6, 0.9]:
        miss_data = data_processor.gen_miss_values(missing_rate)
        inpute_values, class_values = data_processor.inpute_data("MICE")
        
        # Reset indices
        inpute_values_y = inpute_values["Y"].reset_index(drop=True)
        class_values = class_values.reset_index(drop=True)

        # Reduce the length of inpute_values_y and class_values to the same length
        min_length = min(len(inpute_values_y), len(class_values))
        inpute_values_y = inpute_values_y[:min_length]
        class_values = class_values[:min_length]

        comparison_col = [1 if x == y else 0 for x, y in zip(inpute_values_y, class_values)]
        df = pd.DataFrame({"Y": inpute_values_y, "Class": class_values, "Comparison": comparison_col})
        Y = inpute_values["Y"]
        X = inpute_values.drop(["Y"], axis=1)
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=42)

        y_pred = model.predict(X_test)
        df = pd.DataFrame(y_pred, index=X_test.index)
        df['larger_value'] = df.apply(lambda row: 1 if row[0] > row[1] else 0 if row[0] < row[1] else 0.5, axis=1)
        y_pred = df["larger_value"]

        result = pd.DataFrame()
        result["y_pred"] = y_pred
        result["y_test"] = y_test[X_test.index].values
        threshold = 0.5
        y_pred_binary = np.where(result['y_pred'] > threshold, 1, 0)
        y_test_binary = np.where(result['y_test'] > threshold, 1, 0)
        evaluate = data_processor.evaluate(y_test_binary, y_pred_binary)
        print(evaluate)
        filename = output_prefix + str(missing_rate).replace(".", "_")
        data_processor.save_txt(filename, evaluate)

144/144 [==============================] - 1s 3ms/step
{'confusion_matrix': array([[3399, 1202],
       [   7,    0]], dtype=int64), 'accuracy': 0.7376302083333334, 'precision': array([0.9979448, 0.       ]), 'recall': array([0.73875245, 0.        ]), 'f1_score': array([0.84900712, 0.        ]), 'avg_accuracy': 0.49897240164415735, 'auc': [0.536529325922936, 0.391653988263421]}


C:\Users\Ivan\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


157/157 [==============================] - 1s 3ms/step
{'confusion_matrix': array([[3367, 1628],
       [   6,    0]], dtype=int64), 'accuracy': 0.6732653469306139, 'precision': array([0.99822117, 0.        ]), 'recall': array([0.67407407, 0.        ]), 'f1_score': array([0.80473231, 0.        ]), 'avg_accuracy': 0.4991105840498073, 'auc': [0.6575575575575575, 0.6271271271271271]}
161/161 [==============================] - 1s 3ms/step
{'confusion_matrix': array([[3391, 1737],
       [   6,    0]], dtype=int64), 'accuracy': 0.660498636540709, 'precision': array([0.99823374, 0.        ]), 'recall': array([0.66127145, 0.        ]), 'f1_score': array([0.79554252, 0.        ]), 'avg_accuracy': 0.49911686782455106, 'auc': [0.32543551742069676, 0.4360374414976599]}
162/162 [==============================] - 1s 3ms/step
{'confusion_matrix': array([[3390, 1784],
       [   4,    0]], dtype=int64), 'accuracy': 0.6546929316338355, 'precision': array([0.99882145, 0.        ]), 'recall': array([0.6

C:\Users\Ivan\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


163/163 [==============================] - 0s 3ms/step
{'confusion_matrix': array([[3388, 1804],
       [   7,    0]], dtype=int64), 'accuracy': 0.6516637814964417, 'precision': array([0.99793814, 0.        ]), 'recall': array([0.65254237, 0.        ]), 'f1_score': array([0.7890998, 0.       ]), 'avg_accuracy': 0.49896907216494846, 'auc': [0.5684569667620516, 0.5149405679066696]}


...

163/163 [==============================] - 1s 5ms/step
{'confusion_matrix': array([[3386, 1808],
       [   7,    0]], dtype=int64), 'accuracy': 0.6510286483368583, 'precision': array([0.99793693, 0.        ]), 'recall': array([0.65190605, 0.        ]), 'f1_score': array([0.78863398, 0.        ]), 'avg_accuracy': 0.49896846448570586, 'auc': [0.3931459376203312, 0.4449089608889378]}
  1/163 [..............................] - ETA: 6s

C:\Users\Ivan\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


163/163 [==============================] - 1s 4ms/step
{'confusion_matrix': array([[3394, 1800],
       [   8,    0]], dtype=int64), 'accuracy': 0.6524413687043444, 'precision': array([0.99764844, 0.        ]), 'recall': array([0.65344628, 0.        ]), 'f1_score': array([0.78966961, 0.        ]), 'avg_accuracy': 0.49882422104644325, 'auc': [0.7168608009241433, 0.5336927223719676]}
  1/163 [..............................] - ETA: 9s

C:\Users\Ivan\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


163/163 [==============================] - 1s 4ms/step
{'confusion_matrix': array([[3393, 1803],
       [   6,    0]], dtype=int64), 'accuracy': 0.6522491349480969, 'precision': array([0.99823477, 0.        ]), 'recall': array([0.65300231, 0.        ]), 'f1_score': array([0.7895288, 0.       ]), 'avg_accuracy': 0.49911738746690204, 'auc': [0.5030792917628946, 0.6684629201950218]}


In [23]:
data_processor = DataProcessor()
load_data = data_processor.import_data()
test_size = 0.3  # Konstante Testgröße

for model_number in range(0, 3):  # Modelle 1-3 durchlaufen
    model_name = f"Model_{model_number + 1}"
    output_prefix = f"prediction_ref_kNN_D0_{model_name}_"
    model = data_processor.model(model_number)
    
    for missing_rate in [0.3, 0.6, 0.9]:
        miss_data = data_processor.gen_miss_values(missing_rate)
        inpute_values, class_values = data_processor.inpute_data("kNN")
        
        # Reset indices
        inpute_values_y = inpute_values["Y"].reset_index(drop=True)
        class_values = class_values.reset_index(drop=True)

        # Reduce the length of inpute_values_y and class_values to the same length
        min_length = min(len(inpute_values_y), len(class_values))
        inpute_values_y = inpute_values_y[:min_length]
        class_values = class_values[:min_length]

        comparison_col = [1 if x == y else 0 for x, y in zip(inpute_values_y, class_values)]
        df = pd.DataFrame({"Y": inpute_values_y, "Class": class_values, "Comparison": comparison_col})
        
        Y = inpute_values["Y"]
        X = inpute_values.drop(["Y"], axis=1)
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=42)

        y_pred = model.predict(X_test)
        df = pd.DataFrame(y_pred, index=X_test.index)
        df['larger_value'] = df.apply(lambda row: 1 if row[0] > row[1] else 0 if row[0] < row[1] else 0.5, axis=1)
        y_pred = df["larger_value"]

        result = pd.DataFrame()
        result["y_pred"] = y_pred
        result["y_test"] = y_test[X_test.index].values
        threshold = 0.5
        y_pred_binary = np.where(result['y_pred'] > threshold, 1, 0)
        y_test_binary = np.where(result['y_test'] > threshold, 1, 0)
        evaluate = data_processor.evaluate(y_test_binary, y_pred_binary)
        print(evaluate)
        filename = output_prefix + str(missing_rate).replace(".", "_")
        data_processor.save_txt(filename, evaluate)

144/144 [==============================] - 1s 3ms/step
{'confusion_matrix': array([[3398, 1203],
       [   7,    0]], dtype=int64), 'accuracy': 0.7374131944444444, 'precision': array([0.9979442, 0.       ]), 'recall': array([0.7385351, 0.       ]), 'f1_score': array([0.84886335, 0.        ]), 'avg_accuracy': 0.4989720998531571, 'auc': [0.45114416120719103, 0.45803707268606203]}
157/157 [==============================] - 1s 3ms/step
{'confusion_matrix': array([[3367, 1628],
       [   6,    0]], dtype=int64), 'accuracy': 0.6732653469306139, 'precision': array([0.99822117, 0.        ]), 'recall': array([0.67407407, 0.        ]), 'f1_score': array([0.80473231, 0.        ]), 'avg_accuracy': 0.4991105840498073, 'auc': [0.5579245912579246, 0.35722389055722387]}
161/161 [==============================] - 0s 3ms/step
{'confusion_matrix': array([[3391, 1737],
       [   6,    0]], dtype=int64), 'accuracy': 0.660498636540709, 'precision': array([0.99823374, 0.        ]), 'recall': array([0.6612

In [25]:
data_processor = DataProcessor()
load_data = data_processor.import_data()
test_size = 0.3  # Konstante Testgröße

for model_number in range(0, 3):  # Modelle 1-3 durchlaufen
    model_name = f"Model_{model_number + 1}"
    output_prefix = f"prediction_ref_RF_D0_{model_name}_"
    model = data_processor.model(model_number)
    
    for missing_rate in [0.3, 0.6, 0.9]:
        miss_data = data_processor.gen_miss_values(missing_rate)
        inpute_values, class_values = data_processor.inpute_data("RF")
        
        # Reset indices
        inpute_values_y = inpute_values["Y"].reset_index(drop=True)
        class_values = class_values.reset_index(drop=True)

        # Reduce the length of inpute_values_y and class_values to the same length
        min_length = min(len(inpute_values_y), len(class_values))
        inpute_values_y = inpute_values_y[:min_length]
        class_values = class_values[:min_length]

        comparison_col = [1 if x == y else 0 for x, y in zip(inpute_values_y, class_values)]
        df = pd.DataFrame({"Y": inpute_values_y, "Class": class_values, "Comparison": comparison_col})
        
        Y = inpute_values["Y"]
        X = inpute_values.drop(["Y"], axis=1)
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=42)

        y_pred = model.predict(X_test)
        df = pd.DataFrame(y_pred, index=X_test.index)
        df['larger_value'] = df.apply(lambda row: 1 if row[0] > row[1] else 0 if row[0] < row[1] else 0.5, axis=1)
        y_pred = df["larger_value"]

        result = pd.DataFrame()
        result["y_pred"] = y_pred
        result["y_test"] = y_test[X_test.index].values
        threshold = 0.5
        y_pred_binary = np.where(result['y_pred'] > threshold, 1, 0)
        y_test_binary = np.where(result['y_test'] > threshold, 1, 0)
        evaluate = data_processor.evaluate(y_test_binary, y_pred_binary)
        print(evaluate)
        filename = output_prefix + str(missing_rate).replace(".", "_")
        data_processor.save_txt(filename, evaluate)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
144/144 [==============================] - 0s 2ms/step
{'confusion_matrix': array([[3399, 1202],
       [   7,    0]], dtype=int64), 'accuracy': 0.7376302083333334, 'precision': array([0.9979448, 0.       ]), 'recall': array([0.73875245, 0.        ]), 'f1_score': array([0.84900712, 0.        ]), 'avg_accuracy': 0.49897240164415735, 'auc': [0.37550842984444377, 0.5091129257614804]}
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
157/157 [==============================] - 1s 3ms/step
{'confusion_matrix': array([[3367, 1628],
       [   6,    0]], dtype=int64), 'accuracy': 0.6732653469306139, 'precision': array([0.99822117, 0.        ]), 'recall': array([0.67407407, 0.        ]), 'f1_score': array([0.80473231, 0.        ]), 'avg_accuracy': 0.4991105840498073, 'auc': [0.5068401735068402, 0.6666332999666333]}
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
161/161 [==============================] - 0s 3ms/